In [ ]:
# Script to extract and process multiwell fluorescence data from Synergy 2 timelapse experiments.
# It reads .xlsx files, cleans and formats data, and exports it to .csv and .xlsx files in long and wide formats,
# with the option to calculate normalization based on initial fluorescence (F/F0).

from openpyxl import load_workbook
import pandas as pd

# ==== USER-DEFINED PARAMETERS ====
# Change to file number
filename = '20250708_NaG.xlsx'

# Change to file location
location = '/Users/tomas/Desktop/PosDoc/Synergy/CatSper/20250708_NaG'

# Change to experiment date (used for export filenames)
date = 20250708

# Time between reads (depends on number of wells) in seconds
tdelta = 10 

# Time (in minutes) to define "baseline" (i.e., before stimulus)
jump = 2

In [ ]:
# ==== LOAD EXCEL FILE ====
file = f'{location}/{filename}'
wb = load_workbook(file, data_only=True)
ws = wb.active

# ==== DETECT NUMBER OF TIME BLOCKS ====
plate_rows = list("ABCDEFGH")
plate_cols = list(range(1, 13))
block_height = 9      # 8 rows + 1 spacing
block_spacing = 1
row_offset_temp = 2
total_blocks = 0

while True:
    if ws.cell(row=row_offset_temp + 1, column=2).value is None:
        break
    total_blocks += 1
    row_offset_temp += block_height + block_spacing

# ==== BUILD TIME VECTOR ====
timepoints = [i * tdelta for i in range(total_blocks)]

# ==== EXTRACT DATA ====
data = []
row_offset = 2
timepoint = 0

while True:
    if ws.cell(row=row_offset + 1, column=2).value is None:
        break
    for i, row_letter in enumerate(plate_rows):
        for j, col_num in enumerate(plate_cols):
            cell_row = row_offset + 1 + i + 1
            cell_col = 2 + j
            value = ws.cell(row=cell_row, column=cell_col).value
            well = f"{row_letter}{col_num}"
            data.append({
                "Time": timepoints[timepoint],
                "Well": well,
                "Value": value
            })
    row_offset += block_height + block_spacing
    timepoint += 1

# ==== PROCESS DATA ====
synergy = pd.DataFrame(data)
synergy['Row'] = synergy['Well'].str.extract(r'([A-H])')
synergy['Column'] = synergy['Well'].str.extract(r'(\d+)').astype(int)
synergy = synergy.sort_values(by=['Row', 'Column', 'Time']).reset_index(drop=True)
synergy = synergy.drop(columns=['Row', 'Column'])
synergy = synergy[synergy["Value"].notna()]

# ==== PIVOT TO WIDE FORMAT ====
synergy_wide = synergy.pivot(index="Time", columns="Well", values="Value").reset_index()

# ==== CALCULATE NORMALIZED FLUORESCENCE (F/F0) ====
wells = synergy.Well.unique()
for well in wells:
    subset = synergy[synergy.Well == well].copy()
    baseline = subset[subset.Time < jump * 60].Value.mean()
    subset[f'{well} norm'] = subset.Value / baseline
    norm_cols = subset[['Time', f'{well} norm']]
    synergy_wide = pd.merge(synergy_wide, norm_cols, on='Time', how='left')

# ==== EXPORT FILES ====
# Uncomment to export
synergy_wide.to_csv(f'{location}/{date}_Synergy_wide.csv', index=False)
synergy_wide.to_excel(f'{location}/{date}_Synergy_wide.xlsx', index=False)
synergy.to_csv(f'{location}/{date}_Synergy_long.csv', index=False)
synergy.to_excel(f'{location}/{date}_Synergy_long.xlsx', index=False)